In [ ]:
# math
import numpy as np
import pandas as pd

from experiments_2022 import IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    base,
    cleaning,
    viz,
    regression_functions,
)
from experiments_2022.datasets import (
    pull_from_dataset,
)

from utils import (
    PROJECTS_2022_TOTAL,
    PROJECTS_2022,
    NO_WEEKENDS,
    SUMMER_START_2022,
    SUMMER_END_2022,
    DOMINANT_COLORS,
    SINGLE_PLOT_TXT_SIZE,
    SINGLE_PLOT_LEGEND_SIZE,
    SINGLE_PLOT_ANNOTATION_SIZE,
    MULTI_PLOT_TXT_SIZE,
    MULTI_PLOT_TITLE_SIZE,
    MULTI_PLOT_LEGEND_SIZE,
    MULTI_PLOT_ANNOTATION_SIZE,
    add_total_to_results_dict,
    add_total_to_dfs,
)

In [ ]:
DOMINANT_ZONES = {}
for project in PROJECTS_2022:
    dzs = pd.read_csv(f"./dicts_dfs/dominant/{project}.csv").set_index("Unnamed: 0")
    dzs.index.name = None
    DOMINANT_ZONES[project] = dzs

In [ ]:
DOMINANT_ZONES_TOT = add_total_to_results_dict(DOMINANT_ZONES)

In [ ]:
EXCLUDED_ZONES = {}
for project in PROJECTS_2022:
    ezs = pd.read_csv(f"./dicts_dfs/excluded/{project}.csv").set_index("Unnamed: 0")
    ezs.index.name = None
    EXCLUDED_ZONES[project] = ezs

In [ ]:
EXCLUDED_ZONES_TOT = add_total_to_results_dict(EXCLUDED_ZONES)

# Rogue, dominant, dominated

In [ ]:
results_detailed = {}
results_detailed_total = {}

In [ ]:
for project in PROJECTS_2022_TOTAL:
    these_results = pd.DataFrame(
        index=["Included", "Excluded"], columns=["Rogue", "Dominant", "Dominated"]
    )

    these_ezs = EXCLUDED_ZONES_TOT[project]
    these_dzs = DOMINANT_ZONES_TOT[project]
    common = list(set(list(these_ezs.index)).intersection(set(list(these_dzs.index))))
    these_ezs = these_ezs.loc[common, :]
    these_dzs = these_dzs.loc[common, :]

    included = list(these_ezs[these_ezs == 0].dropna().index)
    excluded = list(these_ezs[these_ezs == 1].dropna().index)
    dominated = list(these_dzs[these_dzs == 0].dropna().index)
    dominant = list(these_dzs[these_dzs == 1].dropna().index)
    rogue = list(these_dzs[these_dzs == 2].dropna().index)

    these_results.loc["Included", "Dominated"] = len(
        set(included).intersection(set(dominated))
    )
    these_results.loc["Included", "Dominant"] = len(
        set(included).intersection(set(dominant))
    )
    these_results.loc["Included", "Rogue"] = len(set(included).intersection(set(rogue)))

    these_results.loc["Excluded", "Dominated"] = len(
        set(excluded).intersection(set(dominated))
    )
    these_results.loc["Excluded", "Dominant"] = len(
        set(excluded).intersection(set(dominant))
    )
    these_results.loc["Excluded", "Rogue"] = len(set(excluded).intersection(set(rogue)))

    these_results.loc["Included", :] = these_results.loc["Included", :] / len(included)
    try:
        these_results.loc["Excluded", :] = these_results.loc["Excluded", :] / len(
            excluded
        )
    except:
        these_results.loc["Excluded", :] = np.nan
    results_detailed[project] = these_results

    # based on total zones in building
    these_results_total = pd.DataFrame(
        index=["Included", "Excluded"], columns=["Rogue", "Dominant", "Dominated"]
    )
    these_results_total.loc["Included", "Dominated"] = len(
        set(included).intersection(set(dominated))
    )
    these_results_total.loc["Included", "Dominant"] = len(
        set(included).intersection(set(dominant))
    )
    these_results_total.loc["Included", "Rogue"] = len(
        set(included).intersection(set(rogue))
    )

    these_results_total.loc["Excluded", "Dominated"] = len(
        set(excluded).intersection(set(dominated))
    )
    these_results_total.loc["Excluded", "Dominant"] = len(
        set(excluded).intersection(set(dominant))
    )
    these_results_total.loc["Excluded", "Rogue"] = len(
        set(excluded).intersection(set(rogue))
    )

    these_results_total.loc["Included", :] = these_results_total.loc["Included", :] / (
        len(included) + len(excluded)
    )
    these_results_total.loc["Excluded", :] = these_results_total.loc["Excluded", :] / (
        len(included) + len(excluded)
    )
    results_detailed_total[project] = these_results_total

In [ ]:
100 * results_detailed["TOTAL"]

In [ ]:
100 * results_detailed_total["TOTAL"]

In [ ]:
fig = viz.make_bar_plot(
    y_data=results_detailed["TOTAL"],
    y_range=[0, 1],
    y_axis_title="Fraction of Zones [Unitless]",
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        },
    },
    annotations=["Dominated", "Dominant"],
    annotation_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure10A.png")

In [ ]:
fig = viz.make_bar_plot(
    y_data=results_detailed,
    y_range=[0, 1],
    y_axis_title="Fraction of Zones<br>[Unitless]",
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        },
    },
    annotations=["Dominated", "Dominant"],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
    annotation_size=MULTI_PLOT_ANNOTATION_SIZE,
    vertical_spacing=0.075,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.025,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureE1.png")

# CRs

In [ ]:
# grab and clean data
CRs = pull_from_dataset(
    "2022",
    PROJECTS_2022,
    "zone-simple_cooling_requests",
)

zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=CRs[project]
    )

control_filter = cleaning.create_sp_filter(
    zonal_schedules, sps=[74], reverse_filter=False
)

control_CRs = cleaning.clean_dfs(
    dfs=CRs,
    this_var="zone-dummy",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=True,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    df_filter=control_filter,
)

In [ ]:
control_CRs_tot = add_total_to_dfs(control_CRs)

In [ ]:
all_results = {}

In [ ]:
for project in PROJECTS_2022_TOTAL:
    results = pd.DataFrame(index=["Included", "Excluded"], columns=["CR Rate"])

    # grab project
    these_ezs = EXCLUDED_ZONES_TOT[project]
    these_control_CRS = control_CRs_tot[project]

    # common
    common = list(
        set(list(these_ezs.index)).intersection(set(these_control_CRS.columns))
    )
    these_ezs = these_ezs.loc[common, :]
    these_control_CRS = these_control_CRS.loc[:, common]

    # excluded, includes zone list
    these_ezs_list = list(these_ezs[these_ezs == 1].dropna().index)
    these_izs_list = list(these_ezs[these_ezs == 0].dropna().index)

    # split dfs
    these_exluded_control_CRS = these_control_CRS[these_ezs_list]
    these_included_control_CRS = these_control_CRS[these_izs_list]

    # results
    results.loc["Included", "CR Rate"] = these_included_control_CRS.mean().mean()
    results.loc["Excluded", "CR Rate"] = these_exluded_control_CRS.mean().mean()
    all_results[project] = results.fillna(0)

In [ ]:
fig = viz.make_bar_plot(
    y_data=all_results["TOTAL"],
    y_axis_title="Fraction of Time<br>Sending Cooling Requests<br>Control Days [Unitless]",
    y_range=[0, 0.2],
    dont_add_to_legend=["CR Rate"],
    bar_legend={"color": {"CR Rate": "Black"}},
    annotations=["CR Rate"],
    annotation_thresh=0,
    annotation_size=SINGLE_PLOT_ANNOTATION_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure10B.png")

In [ ]:
fig = viz.make_bar_plot(
    y_data=all_results,
    y_axis_title="Fraction of Time<br>Sending Cooling Requests<br>Control Days [Unitless]",
    y_range=[0, 0.5],
    dont_add_to_legend=["CR Rate"],
    bar_legend={"color": {"CR Rate": "Black"}},
    annotations=["CR Rate"],
    annotation_thresh=0,
    horizontal_spacing=0.125,
    vertical_spacing=0.075,
    annotation_size=MULTI_PLOT_TXT_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureE2.png")

# T - CSP

In [ ]:
# grab and clean data
deviation = pull_from_dataset(
    "2022",
    PROJECTS_2022,
    "zone-deviation_coolsp",
)

deviation = cleaning.clean_dfs(
    dfs=deviation,
    this_var="zone-deviation_coolsp",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=True,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)

zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=deviation[project]
    )

control_filter = cleaning.create_sp_filter(
    zonal_schedules, sps=[74], reverse_filter=False
)

control_deviation = cleaning.clean_dfs(
    dfs=deviation,
    this_var="zone-deviation_coolsp",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=True,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    df_filter=control_filter,
)

In [ ]:
control_deviation_tot = add_total_to_dfs(control_deviation)

In [ ]:
all_results = {}

In [ ]:
for project in PROJECTS_2022_TOTAL:
    results = pd.DataFrame(index=["Included", "Excluded"], columns=["T - CSP"])

    # grab project
    these_ezs = EXCLUDED_ZONES_TOT[project]
    these_control_deviation = control_deviation_tot[project]

    # common
    common = list(
        set(list(these_ezs.index)).intersection(set(these_control_deviation.columns))
    )
    these_ezs = these_ezs.loc[common, :]
    these_control_deviation = these_control_deviation.loc[:, common]

    # excluded, includes zone list
    these_ezs_list = list(these_ezs[these_ezs == 1].dropna().index)
    these_izs_list = list(these_ezs[these_ezs == 0].dropna().index)

    # split dfs
    these_exluded_control_deviation = these_control_deviation[these_ezs_list]
    these_included_control_deviation = these_control_deviation[these_izs_list]

    # results
    results.loc["Included", "T - CSP"] = these_included_control_deviation.mean().mean()
    results.loc["Excluded", "T - CSP"] = these_exluded_control_deviation.mean().mean()
    all_results[project] = results.fillna(0)

In [ ]:
fig = viz.make_bar_plot(
    y_data=all_results["TOTAL"],
    y_axis_title="T - CSP [°C]<br>Control Days",
    y_range=[-2, 0],
    dont_add_to_legend=["T - CSP"],
    bar_legend={"color": {"T - CSP": "Black"}},
    annotations=["T - CSP"],
    annotation_thresh=-2,
    horizontal_spacing=0.125,
    vertical_spacing=0.075,
    annotation_size=SINGLE_PLOT_ANNOTATION_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure10C.png")

In [ ]:
fig = viz.make_bar_plot(
    y_data=all_results,
    y_axis_title="T - CSP [°C]<br>Control Days",
    horizontal_spacing=0.1,
    vertical_spacing=0.075,
    y_range=[-3, 0],
    dont_add_to_legend=["T - CSP"],
    bar_legend={"color": {"T - CSP": "Black"}},
    annotations=["T - CSP"],
    annotation_thresh=-3,
    annotation_size=MULTI_PLOT_TXT_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureE3.png")